## Clickhouse-based Index

Good for aggregations, maybe less so for large search.

In [1]:
!pip install clickhouse-driver[lz4]

You should consider upgrading via the '/home/pq/UTK/NSDF/NSDF_Object-Catalog_Paper/nsdf-data-catalog/venv/bin/python3 -m pip install --upgrade pip' command.


## Connect to Clickhouse

In [2]:
from clickhouse_driver import Client

In [3]:
client = Client('localhost')

In [4]:
result = client.execute('SELECT now(), version()')
result

[(datetime.datetime(2022, 7, 11, 1, 41, 37), '22.1.3.7')]

In [5]:
print("RESULT: {0}: {1}".format(type(result), result))

RESULT: <class 'list'>: [(datetime.datetime(2022, 7, 11, 1, 41, 37), '22.1.3.7')]


In [6]:
client.execute('SHOW DATABASES')

[('INFORMATION_SCHEMA',), ('default',), ('information_schema',), ('system',)]

In [7]:
client.execute('SHOW TABLES')

[('nsdf_catalog_index',)]

In [8]:
client.execute('DROP TABLE IF EXISTS nsdf_catalog_index')
client.execute("""
CREATE TABLE nsdf_catalog_index (
    id String,
    name String,
    size Int64,
    origin_id String,
    origin_uri String,
    replicas Array(String),
    tags Array(String)
) ENGINE = Memory""")
client.execute('SHOW TABLES IN default')

[('nsdf_catalog_index',)]

## Prepare some Entries

In [9]:
import uuid

In [10]:
entries = {}
for i in range(0,100):
    entry = {
        "id": str(uuid.uuid4()), 
        "name": f"Frozen Yogurt ({i})" ,
        "size": i,
        "origin_uri": "",
        "origin_id":"Kuehlregal",
        "replicas": ["rep1", "rep2"],
        "tags":["abc","def"]
    }
    entries[entry['id']] = entry

In [11]:
entry

{'id': 'cf4c1d65-0160-4162-acb4-5ad4753215ba',
 'name': 'Frozen Yogurt (99)',
 'size': 99,
 'origin_uri': '',
 'origin_id': 'Kuehlregal',
 'replicas': ['rep1', 'rep2'],
 'tags': ['abc', 'def']}

In [12]:
client.execute('DESCRIBE nsdf_catalog_index')

[('id', 'String', '', '', '', '', ''),
 ('name', 'String', '', '', '', '', ''),
 ('size', 'Int64', '', '', '', '', ''),
 ('origin_id', 'String', '', '', '', '', ''),
 ('origin_uri', 'String', '', '', '', '', ''),
 ('replicas', 'Array(String)', '', '', '', '', ''),
 ('tags', 'Array(String)', '', '', '', '', '')]

In [13]:
client.execute(
    'INSERT INTO nsdf_catalog_index (id, name, size, origin_id, origin_uri, replicas, tags) VALUES',
    list(entries.values())
)

100

In [14]:
client.execute('SELECT * FROM nsdf_catalog_index LIMIT 3')

[('ccdbcfbb-c47f-4993-a582-49f829d86c80',
  'Frozen Yogurt (0)',
  0,
  'Kuehlregal',
  '',
  ['rep1', 'rep2'],
  ['abc', 'def']),
 ('9b239a22-5739-4d63-8874-4b0b753e6810',
  'Frozen Yogurt (1)',
  1,
  'Kuehlregal',
  '',
  ['rep1', 'rep2'],
  ['abc', 'def']),
 ('49893e90-dac1-47a0-b892-f2d1abe8b68d',
  'Frozen Yogurt (2)',
  2,
  'Kuehlregal',
  '',
  ['rep1', 'rep2'],
  ['abc', 'def'])]